In [2]:
import os
import base64
import requests
import pandas as pd
from io import StringIO

In [3]:
api_key = os.getenv("OPENAI_API_KEY")

In [4]:
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

def explain_image_base64(question, base64_image):

    headers = {
      "Content-Type": "application/json",
      "Authorization": f"Bearer {api_key}"
    }

    payload = {
      "model": "gpt-4-vision-preview",
      "messages": [
        {
          "role": "user",
          "content": [
            {
              "type": "text",
              "text": question
            },
            {
              "type": "image_url",
              "image_url": {
                "url": f"data:image/jpeg;base64,{base64_image}"
              }
            }
          ]
        }
      ],
      "max_tokens": 1000
    }

    openai_url = "https://api.openai.com/v1/chat/completions"
    
    response = requests.post(openai_url, headers=headers, json=payload)
    return response.json()

def explain_image(question, image_path):
    base64_image = encode_image(image_path)
    return explain_image_base64(question, base64_image)

In [12]:
image_path = r"D:\1\Image.jpg"
json_answer = explain_image("""
Create csv file from this image with column 3 columns.
In first column 'Latin Content' collect all about Latin content.
In second column 'Gender' place gender of word (m/f/n).
In last column 'German Translations' collect transtaltions.
Be aware that some rows are places in 2 lines - please combine it correctly based on gray line seperators.
User separator |. Be sure that csv has exactly 3 columns.
""", image_path)
answer = json_answer['choices'][0]['message']['content']
print(answer)

Latin Content|Gender|German Translations
ingenium|n|die Begabung, das Talent, der Verstand
studium|n|die Beschäftigung, das Engagement, das Interesse
manifestus, a, um|m/f/n|offenkundig; überführt
quantus, a, um|m/f/n|wie groß, wie viel
ingredī, ingredior, ingressus sum|m|betreten; beginnen
ambitio, ambitionis|f|der Ehrgeiz
scilicet Adv.|n|freilich, natürlich
vīdērī, videor, visus sum|m|scheinen, gelten (als)
instruere, instruō, īnstrūxī, īnstrūctum|m|aufstellen, ausrüsten; unterrichten
cōnfiterī, cōnfiteor, cōnfessus sum|m|(ein)gestehen
cōnsuēscere, cōnsuēscō, cōnsuēvī, cōnsuētum|m|sich daran gewöhnen
Equus pārēre cōnsuēscit.||Das Pferd gewöhnt sich daran zu gehorchen.
remittēre, remittō, remīsī, remissum|m|zurückschicken; nachlassen, vermindern
status, statūs|m|der Stand, die Lage; Pl. die Umstände
āctiō, āctiōnis|f|die Tätigkeit; die Gerichtsverhandlung; die Rede
onus, oneris|n|die Last
suscipere, suscipiō, suscēpī, susceptum|m|auf sich nehmen, sich (einer Sache) annehmen, unternehm

In [13]:
lines = answer.split('\n')
lines

['Latin Content|Gender|German Translations',
 'ingenium|n|die Begabung, das Talent, der Verstand',
 'studium|n|die Beschäftigung, das Engagement, das Interesse',
 'manifestus, a, um|m/f/n|offenkundig; überführt',
 'quantus, a, um|m/f/n|wie groß, wie viel',
 'ingredī, ingredior, ingressus sum|m|betreten; beginnen',
 'ambitio, ambitionis|f|der Ehrgeiz',
 'scilicet Adv.|n|freilich, natürlich',
 'vīdērī, videor, visus sum|m|scheinen, gelten (als)',
 'instruere, instruō, īnstrūxī, īnstrūctum|m|aufstellen, ausrüsten; unterrichten',
 'cōnfiterī, cōnfiteor, cōnfessus sum|m|(ein)gestehen',
 'cōnsuēscere, cōnsuēscō, cōnsuēvī, cōnsuētum|m|sich daran gewöhnen',
 'Equus pārēre cōnsuēscit.||Das Pferd gewöhnt sich daran zu gehorchen.',
 'remittēre, remittō, remīsī, remissum|m|zurückschicken; nachlassen, vermindern',
 'status, statūs|m|der Stand, die Lage; Pl. die Umstände',
 'āctiō, āctiōnis|f|die Tätigkeit; die Gerichtsverhandlung; die Rede',
 'onus, oneris|n|die Last',
 'suscipere, suscipiō, suscēp

In [39]:
df = pd.read_csv(StringIO(answer), delimiter='|', names = ['L', 'G', 'D'], header = 0)

In [40]:
df

,L,G,D
0,ingenium,n,"die Begabung, das Talent, der Verstand"
1,studium,n,"die Beschäftigung, das Engagement, das Interesse"
2,"manifestus, a, um",m/f/n,offenkundig; überführt
3,"quantus, a, um",m/f/n,"wie groß, wie viel"
4,"ingredī, ingredior, ingressus sum",m,betreten; beginnen
5,"ambitio, ambitionis",f,der Ehrgeiz
6,scilicet Adv.,n,"freilich, natürlich"
7,"vīdērī, videor, visus sum",m,"scheinen, gelten (als)"
8,"instruere, instruō, īnstrūxī, īnstrūctum",m,"aufstellen, ausrüsten; unterrichten"
9,"cōnfiterī, cōnfiteor, cōnfessus sum",m,(ein)gestehen


In [50]:
df['result'] = df.apply(lambda row:
    f'{row["L"]}<i>m.Gen</i><br><i><span style="color: rgb(170, 170, 0);">{row["G"]}</span></i>|{row["D"]}<br><span style="color: rgb(170, 170, 0);">', 
    axis=1)



In [51]:
result_str = '\n'.join(df['result'])

In [52]:
print(result_str)

ingenium<i>m.Gen</i><br><i><span style="color: rgb(170, 170, 0);">n</span></i>|die Begabung, das Talent, der Verstand<br><span style="color: rgb(170, 170, 0);">
studium<i>m.Gen</i><br><i><span style="color: rgb(170, 170, 0);">n</span></i>|die Beschäftigung, das Engagement, das Interesse<br><span style="color: rgb(170, 170, 0);">
manifestus, a, um<i>m.Gen</i><br><i><span style="color: rgb(170, 170, 0);">m/f/n</span></i>|offenkundig; überführt<br><span style="color: rgb(170, 170, 0);">
quantus, a, um<i>m.Gen</i><br><i><span style="color: rgb(170, 170, 0);">m/f/n</span></i>|wie groß, wie viel<br><span style="color: rgb(170, 170, 0);">
ingredī, ingredior, ingressus sum<i>m.Gen</i><br><i><span style="color: rgb(170, 170, 0);">m</span></i>|betreten; beginnen<br><span style="color: rgb(170, 170, 0);">
ambitio, ambitionis<i>m.Gen</i><br><i><span style="color: rgb(170, 170, 0);">f</span></i>|der Ehrgeiz<br><span style="color: rgb(170, 170, 0);">
scilicet Adv.<i>m.Gen</i><br><i><span style="colo